In [1]:
import os
import random
random.seed(99)
import shutil

import numpy as np
np.random.seed(99)

In [2]:
IS_RESET = True
SRC_DATA = './dataset/img_data_152'
DST_DATA = './federated_data'
CLIENTS = 5 + 1 # 1 is validation set (index 0)
# balanced_iid balanced_noniid unbalanced_iid unbalanced_noniid
TYPE = 'as_is'

In [3]:
def get_files(root_dir, ext=('.jpg', '.png')):
    queue = [root_dir]
    while len(queue) != 0:
        nest_dir = queue.pop()
        label = None
        files = list()
        with os.scandir(nest_dir) as it:
            for entry in it:
                if not entry.name.startswith('.') and entry.is_file():
                    if entry.name.endswith(ext):
                        label = os.path.basename(os.path.dirname(entry.path)) # dirname is label
                        files.append(entry.path)
                elif not entry.name.startswith('.') and entry.is_dir():
                    queue.append(entry.path)
            if label is not None:
                yield label, files

In [4]:
data_root = os.path.abspath(os.path.expanduser(SRC_DATA))
dst_base = os.path.abspath(os.path.expanduser(DST_DATA))

if IS_RESET:
    if os.path.exists(dst_base):
        shutil.rmtree(dst_base)

if TYPE is 'balanced_iid':
    cnt = [178]*6
    for label, files in get_files(data_root):
        random.shuffle(files)
        for i in range(0, CLIENTS):
            dst = os.path.join(dst_base, str(i), label)
            os.makedirs(dst, exist_ok=True) 
            for path in files[sum(cnt[:i]):sum(cnt[:i+1])]:
                shutil.copy(path, dst)
elif TYPE is 'unbalanced_iid':
    cnt = [178, 59, 118, 178, 238, 297]
    for label, files in get_files(data_root):
        random.shuffle(files)
        for i in range(0, CLIENTS):
            dst = os.path.join(dst_base, str(i), label)
            os.makedirs(dst, exist_ok=True) 
            for path in files[sum(cnt[:i]):sum(cnt[:i+1])]:
                shutil.copy(path, dst)
elif TYPE is 'balanced_noniid':
    class_div = dict()
    with os.scandir(data_root) as it:
        classes = list()
        for entry in it:
            if not entry.name.startswith('.') and entry.is_dir():
                name = os.path.basename(entry.path)
                classes.append(name)
        for _ in range(2):
            random.shuffle(classes)
            for div in classes:
                for i in range(1, CLIENTS):
                    l = class_div.get(i, set())
                    if len(l) == 6:
                        continue
                    elif div in l:
                        continue
                    l.add(div)
                    class_div[i] = l
                    break
    cnt = [178, 445, 445]
    appear = list()
    for label, files in get_files(data_root):
        random.shuffle(files)
        for i in range(0, CLIENTS):
            if i != 0:
                if label not in class_div[i]:
                    continue
            dst = os.path.join(dst_base, str(i), label)
            os.makedirs(dst, exist_ok=True)
            s = appear.count(label)
            for path in files[sum(cnt[:s]):sum(cnt[:s+1])]:
                shutil.copy(path, dst)
            appear.append(label)
elif TYPE is 'unbalanced_noniid':
    class_div = dict()
    with os.scandir(data_root) as it:
        classes = list()
        for entry in it:
            if not entry.name.startswith('.') and entry.is_dir():
                name = os.path.basename(entry.path)
                classes.append(name)
        for _ in range(2):
            random.shuffle(classes)
            for div in classes:
                for i in range(1, CLIENTS):
                    l = class_div.get(i, set())
                    if len(l) == 6:
                        continue
                    elif div in l:
                        continue
                    l.add(div)
                    class_div[i] = l
                    break
    streak = dict()
    nums = dict()
    for i in range(1, 6):
        nums[i] = [356]*i + [534]*(6-i)
        random.shuffle(nums[i])
    for label, files in get_files(data_root):
        random.shuffle(files)
        dst = os.path.join(dst_base, str(0), label)
        os.makedirs(dst, exist_ok=True)
        for path in files[0:178]:
            shutil.copy(path, dst)
        streak[label] = streak.get(label, 0) + 178
        for i in range(1, CLIENTS):
            if label not in class_div[i]:
                continue
            dst = os.path.join(dst_base, str(i), label)
            os.makedirs(dst, exist_ok=True)
            s = streak.get(label, 0)
            e = nums[i].pop()
            for path in files[s:s+e]:
                shutil.copy(path, dst)
            streak[label] = s + e
elif TYPE is 'as_is':
    for label, files in get_files(data_root):
        random.shuffle(files)
        cnt = int(len(files)*0.01)
        for i in range(0, CLIENTS):
            dst = os.path.join(dst_base, str(i), label)
            os.makedirs(dst, exist_ok=True) 
            for path in files[i*cnt:(i+1)*cnt]:
                shutil.copy(path, dst)

In [7]:
class_div = dict()
with os.scandir(data_root) as it:
    classes = list()
    for entry in it:
        if not entry.name.startswith('.') and entry.is_dir():
            name = os.path.basename(entry.path)
            classes.append(name)
    for _ in range(2):
        random.shuffle(classes)
        for div in classes:
            for i in range(1, CLIENTS):
                l = class_div.get(i, set())
                if len(l) == 6:
                    continue
                elif div in l:
                    continue
                l.add(div)
                class_div[i] = l
                break
import pprint
pprint.pprint(class_div)

{1: {'email', 'spotify', 'vimeo', 'facebook', 'torrent', 'scp'},
 2: {'aim', 'voipbuster', 'ftps', 'netflix', 'skype', 'youtube'},
 3: {'email', 'gmail', 'hangout', 'skype', 'vimeo', 'sftp'},
 4: {'voipbuster', 'gmail', 'ftps', 'spotify', 'scp', 'sftp'},
 5: {'aim', 'hangout', 'netflix', 'youtube', 'facebook', 'torrent'}}


In [ ]:
{1: {'email1', 'spotify1', 'vimeo1', 'facebook1', 'torrent1', 'scp1'},
 2: {'aim1', 'voipbuster1', 'ftps1', 'netflix1', 'skype1', 'youtube1'},
 3: {'email2', 'gmail1', 'hangout1', 'skype2', 'vimeo2', 'sftp1'},
 4: {'voipbuster2', 'gmail2', 'ftps2', 'spotify2', 'scp2', 'sftp2'},
 5: {'aim2', 'hangout2', 'netflix2', 'youtube2', 'facebook2', 'torrent2'}}

In [ ]:
1=> 1, 2
2=> 3, 4, 5